In [ ]:
'''PLEASE SPECIFY YOUR PARAMETERS HERE'''

#number of nodes to place
#please input a PERFECT SQUARE so that the nodes can be distributed evenly
nodesToPlace = 16

#sides of you area to simulate
playgroundSide = 12

#path to your logistic regression results
filePath = "Probability.csv"

'''After the input is done, please press Kernel-->Restart & Run All '''

In [ ]:
import numpy as np
import scipy as sp
from scipy import stats
from scipy import integrate
from sympy.solvers import solve
from sympy import Symbol, N
from sympy import Poly
from sympy.solvers.inequalities import solve_poly_inequality
import matplotlib.pyplot as plt

'''If you don't have matlab engine, please run the matlab code separately'''

# import matlab.engine
# eng = matlab.engine.start_matlab()
# eng.Probability(nargout=0)
# eng.quit()

prior = np.loadtxt(filePath, dtype = float).reshape((10,10))
print (prior)

In [ ]:
class SNode:
    def __init__(self, x, y, ID, lambdai):
        
        
        #initialize off sensor, location and decision thresh
        self.state = -1
        self.Xi = x
        self.Yi = y
        self.id = ID
        self.thresh = lambdai
        
    def distance(self, x1,y1,x2,y2):
        return np.sqrt((x1-x2)**2+(y1-y2)**2)
    
    #polynomial decaying signal
    def signalPoly(self,x, y, b=1, a=4, en=10000):
        return 1000000000000000 if (x==self.Xi and y==self.Yi) else b*en/(np.sqrt((x-self.Xi)**2.0+(y-self.Yi)**2.0)**(a))
    
    #gaussian decay (exponential of squared distance)
    def signal(self, x, y, sigmasqr=4, Tmax=1000):
        return Tmax*np.exp(-self.distance(x, y, self.Xi, self.Yi)**2/(2*sigmasqr))
    
    #exponential-of-inverse-distance decay
    def signal(self, x, y, Tmax=10, Lsqr = 1):
        return Tmax*(1-np.exp(-Lsqr/self.distance(x, y, self.Xi, self.Yi)))
        
    def PDInt(self, msmt, x, y, sigma=1):
        
        #prob density of PFi
        result = (1/(np.sqrt(2*np.pi)*sigma))
        result *= np.exp(-(msmt-np.sqrt(self.signal(x, y)))**2/(2*sigma**2))
        return result
        
    def PDi(self, xsig, ysig, lambdai, sigma=1):
        
        #integrate PFi from the given decision threshold
        self.thresh = lambdai
        res = integrate.quad(self.PDInt, self.thresh, np.inf, args = (xsig, ysig))[0]
        return res
        

In [ ]:
class GridMap:
    def __init__(self, dim, alphathresh, num, std=1.0):
        
        #initialze grid
        self.grid = np.zeros((dim,dim))
        self.size = dim
        
        #initialize decision threshold, std of PFi, decision boundary
        self.AlphaZero = alphathresh
        self.alpha = -100
        self.numNode = num
        self.sigma = std
        self.decisionBound = np.zeros((dim,dim))
        
        #active nodes and how many there are
        self.activeNodeNum = 0
        self.activeNode = []
        
        #initialize nodes
        self.nodeArray = []
        side = int(np.sqrt(self.numNode))
        interval = dim/(side-1)-1
        for i in range(0, side):
            for j in range(0,side):
                self.nodeArray.append(SNode(i*interval, j*interval, i*side+j, self.decisionBound[i*interval,j*interval]))

    def getPF(self, x, y):
        if len(self.activeNode)==0:
            return
        global prob 
        prob = 0.0
        
        self.getPFHelper(x, y, self.activeNode, 1.0, 0, 0)
        return prob
        
    
    def getPFHelper(self, x, y, nodes, term, index, counter):
        if index==len(nodes):
            if counter > int(len(nodes)/2):
                global prob
                prob += term
            return
        alpha0 = prior[x,y]*stats.norm.cdf(self.decisionBound[x,y], loc=0, scale=self.sigma)
        self.getPFHelper(x, y, nodes, term*alpha0, index+1, counter+1)
        self.getPFHelper(x, y, nodes, term*(1.0-alpha0), index+1, counter)

    def solveLambda(self):
        for i in range(self.size):
            for j in range(self.size):
                self.decisionBound[i,j] =(1.0-prior[i,j])/ prior[i,j]
        return prior
    
    def activateNode(self, ID):
        
        print ID
        self.nodeArray[ID].state = 1
        self.activeNode.append(ID)
        self.activeNodeNum += 1
        print self.activeNode
        
        
        self.solveLambda()
        
        maxPF = -np.inf
        
        for i in range(0,self.size):
            for j in range(0, self.size):
                PFi = self.getPF(i, j)
                if (PFi > maxPF):
                    maxPF = PFi
                    
        print maxPF
        return maxPF <= self.AlphaZero

In [ ]:
class CentralControl:
    def __init__(self, alphathresh, betathresh, num, std=1):
        self.grid = GridMap(10, alphathresh, num, std=1)
        self.SysPD = 0
        self.beta = betathresh

    def getPDDFS(self, x, y, nodes):
        if len(nodes)==0:
            return
        global solutions 
        solutions = 0.0
        
        self.getPDHelper(x, y, nodes, 1.0, 0, 0)
        return solutions
    
    def getPDHelper(self, x, y, nodes, term, index, counter):
  
        if index==len(nodes):
            if counter > int(len(nodes)/2):
                global solutions
                solutions += term
            return
        
        termtemp = term*self.grid.nodeArray[nodes[index]].PDi(x, y, self.grid.decisionBound[x,y])
        self.getPDHelper(x, y, nodes, termtemp, index+1, counter+1)
        
        termtemp = term*(1-self.grid.nodeArray[nodes[index]].PDi(x, y, self.grid.decisionBound[x,y]))
        self.getPDHelper(x, y, nodes, termtemp, index+1, counter)

    
    def getPDJS(self, x, y, nodes):
        majority = int(self.grid.activeNodeNum / 2)+1
        denom = np.math.factorial(self.grid.activeNodeNum) / np.math.factorial(majority-1) / np.math.factorial(self.grid.activeNodeNum-majority+1)
        
        logPDi = np.zeros(self.grid.numNode)
        minuslogPDi = np.zeros(self.grid.numNode)
        
        for i in nodes:
            if self.grid.nodeArray[i].state==1:
                logPDi[i] = np.log(self.grid.nodeArray[i].PDi(x,y,self.grid.decisionBound[x,y]))
                minuslogPDi[i] = np.log(1-self.grid.nodeArray[i].PDi(x,y,self.grid.decisionBound[x,y]))
        
        A = 0
        for i in range(majority-1, self.grid.activeNodeNum):
            A += np.math.factorial(self.grid.activeNodeNum-1) / np.math.factorial(i) / np.math.factorial(self.grid.activeNodeNum-1-i)
            
        B = 0
        for i in range(0, majority-1):
            B += np.math.factorial(self.grid.activeNodeNum-1) / np.math.factorial(i) / np.math.factorial(majority-2-i)
        
        print self.grid.activeNodeNum, majority, A, B,
        LB = A*sum(logPDi) + B*sum(minuslogPDi)
        return np.exp(LB/denom + np.log(denom))
        
    def run(self):
        first = np.random.randint(low=0, high=self.grid.numNode)
        grid.activate(first)
        return
    
    def distFromActive(self, x,y):
        minDist = np.inf
        for i in self.grid.activeNode:
            iDist = self.distance(self.grid.nodeArray[i].Xi, self.grid.nodeArray[i].Yi, x, y)
            minDist = iDist if iDist<minDist else minDist
        return minDist
    
    def distance(self, x1,y1,x2,y2):
        return np.sqrt((x1-x2)**2+(y1-y2)**2)
    
    def getNeighborNode(self, x, y):
        MinDist = np.inf
        MinID = -1
        for i in self.grid.nodeArray:
            if (i.state!=1):
                idist = self.distance(x,y,i.Xi, i.Yi) - self.distFromActive(i.Xi, i.Yi)
                if (idist<MinDist):
                    MinDist = idist
                    MinID = i.id
        return MinID
        
        
    def activate(self, SNid):
        eventSig = np.zeros((self.grid.size,self.grid.size))
        self.grid.activateNode(SNid)
        PDMinSig = -1
        
        heatmap = plt.imshow(eventSig, cmap='hot', interpolation='nearest')
        plt.colorbar(heatmap)
        plt.show()
        
        
        PDMinPoint = (-1,-1)
        PDMinSig = np.inf
        for i in range(0,self.grid.size):
            for j in range(0, self.grid.size):
                eventSig[i,j] = self.getPDDFS(i,j,self.grid.activeNode)
                if (eventSig[i,j] < PDMinSig):
                    PDMinPoint = (i,j)
                    PDMinSig = eventSig[i,j]
        


        toTurnOn = self.getNeighborNode(PDMinPoint[0],PDMinPoint[1])
        print toTurnOn

        terminate = self.grid.activateNode(toTurnOn)

        
        while ((not terminate) and PDMinSig <= self.beta):
            heatmap = plt.imshow(eventSig, cmap='hot', interpolation='nearest')
            plt.colorbar(heatmap)
            plt.show()
            
            if (self.grid.activeNodeNum == self.grid.numNode):
                return -1
            
            PDMinPoint = (-1,-1)
            PDMinSig = np.inf
            for i in range(0,self.grid.size):
                for j in range(0, self.grid.size):
                    eventSig[i,j] = self.getPDDFS(i,j,self.grid.activeNode)
                    if (eventSig[i,j] < PDMinSig):
                        PDMinPoint = (i,j)
                        PDMinSig = eventSig[i,j]
            print PDMinSig, "@", PDMinPoint
            
            toTurnOn = self.getNeighborNode(PDMinPoint[0],PDMinPoint[1])
            print toTurnOn
            
                    
            terminate = self.grid.activateNode(toTurnOn)
        heatmap = plt.imshow(eventSig, cmap='hot', interpolation='nearest')
        plt.colorbar(heatmap)
        plt.show()    
        for i in self.grid.nodeArray:
            print i.state
        
        return 0
            

In [ ]:
if __name__ == "__main__":
    control = CentralControl(0.10,0.9,16)
    sig = np.random.randint(0,16)
    control.activate(sig)
    

In [ ]:
heatmap = plt.imshow(prior, cmap='hot', interpolation='nearest')
plt.colorbar(heatmap)
plt.show()